## Import library

In [1]:
import seaborn as sns

import yfinance as yf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import ta 
import plotly.graph_objects as go
import stock_lib as stlib

## Import data

In [2]:
data = yf.download("TSLA", period='60d', interval='5m')
data = data.reset_index()

macd = ta.trend.MACD(close=data['Close'])
bb = ta.volatility.BollingerBands(close=data['Close'])
cmf = ta.volume.ChaikinMoneyFlowIndicator(high=data['High'], low=data['Low'], close=data['Close'], volume=data['Volume'])
rsi = ta.momentum.RSIIndicator(close=data['Close'])

data['rsi'] = rsi.rsi()
data['macd'] = macd.macd_diff()
data['cmf'] = cmf.chaikin_money_flow()
data['bb_width'] = bb.bollinger_wband()

data['close_3'] = data['Close'].diff(periods=-5)
data['close_3'] = (data['close_3'] / data['Close']) * 100

data['close_8'] = data['Close'].diff(periods=-8)
data['close_8'] = (data['close_8'] / data['Close']) * 100

data['close_13'] = data['Close'].diff(periods=-13)
data['close_13'] = (data['close_13'] / data['Close']) * 100
data['close_21'] = data['Close'].diff(periods=-21)
data['close_21'] = (data['close_21'] / data['Close']) * 100

data = data.dropna()
# data1 = data[['rsi', 'rsi_diff', 'close_5', 'close_10', 'macd', 'cmf', 'bb_width']]
data1 = data[['rsi', 'macd', 'cmf', 'bb_width', 'close_3', 'close_8', 'close_13', 'close_21']]

[*********************100%***********************]  1 of 1 completed


In [3]:
# data2 = data1[(data1['close_5'] > 1) | (data1['close_5'] < -1)]
data2 = data1
data2 = data2.reset_index()
data2 = data2.drop('index', axis=1)
data2

,rsi,macd,cmf,bb_width,close_3,close_8,close_13,close_21
0,72.789165,-0.177410,0.221228,2.092604,0.200951,1.001188,1.223042,1.801425
1,69.753067,-0.215517,0.197262,2.028301,0.568225,0.550264,1.038805,1.967740
2,69.711123,-0.266054,0.186139,1.880235,0.683244,0.972035,1.232139,1.361463
3,63.856753,-0.407092,0.168855,1.627332,0.704499,0.650897,1.351636,1.167684
4,56.515958,-0.634296,0.164395,1.350274,0.074355,0.870586,1.361457,0.811388
...,...,...,...,...,...,...,...,...
4619,35.196819,-0.474534,0.239216,0.816744,0.083638,0.253499,-0.041155,-0.824430
4620,43.554171,-0.431766,0.259344,0.785037,0.138295,0.397630,-0.271720,-0.764778
4621,43.161297,-0.388863,0.230933,0.811599,0.473288,0.368514,-0.153755,-0.489137
4622,38.906900,-0.399572,0.172141,0.871302,0.285269,0.265432,-0.254812,-0.787004


In [4]:
label = []

for i in range(0, len(data2)):
    
    if data2['close_13'][i] < -2:
        label.append(0)
    elif data2['close_13'][i] < -1:
        label.append(1)
    elif data2['close_13'][i] < 0:
        label.append(2)
    elif data2['close_13'][i] < 1:
        label.append(3)
    elif data2['close_13'][i] < 2:
        label.append(4)
    else:
        label.append(5)
        
data2['label'] = label

## Create Model

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(random_state=1)

X = data2.drop(['close_3', 'close_8', 'close_13', 'close_21', 'label'], axis=1).to_numpy()
y = data2[['close_3', 'close_8', 'close_13', 'close_21']].to_numpy()

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

(4624, 4)
(4624, 4)


## Regression

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)

clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print(metrics.r2_score(y_test, prediction, multioutput='uniform_average'))
print(metrics.r2_score(y_test, prediction, multioutput='raw_values'))

0.13361268875471818
[0.11781083 0.07811237 0.17074407 0.16778348]


## Classification

In [7]:
from sklearn.ensemble import RandomForestClassifier

clf1 = RandomForestClassifier(random_state=0)

X = data2.drop(['close_5', 'label'], axis=1).to_numpy()
y = data2[['label']].to_numpy()

X = np.array(X)
y = np.array(y).ravel()

print(X.shape)
print(y.shape)

KeyError: "['close_5'] not found in axis"

In [ ]:
scores = cross_val_score(clf1, X, y, cv=10)
scores

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf1.fit(X_train, y_train)

In [ ]:
prediction = clf1.predict(X_test)

print("prediction:   ", prediction)
print()
print("actual value: ", y_test)
print()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf1, X_test, y_test)  
plt.show()

In [ ]:
from sklearn.neural_network import MLPClassifier 

clf = MLPClassifier(random_state=1, 
                    max_iter=1000,
                    hidden_layer_sizes=(10, 100))

scores = cross_val_score(clf, X, y, cv=10)
scores